In [4]:
import pandas as pd
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

In [5]:
df=pd.read_xml(r'D:\Github\ship_data\093 Syrma - AIS Data 1-Jan-24 to 30-Apr-24.xml',parse_dates=["TIMESTAMP"])

In [6]:
df.head()

,MMSI,IMO,STATUS,SPEED,LON,LAT,COURSE,HEADING,TIMESTAMP,SHIP_ID
0,312564000,9218923,5,0,103.8900,1.088333,40,511,2024-01-19 03:16:53,662466
1,312564000,9218923,5,0,103.8900,1.088333,40,77,2024-01-19 03:37:53,662466
2,312564000,9218923,5,0,103.8914,1.089367,40,78,2024-01-19 04:00:48,662466
3,312564000,9218923,5,0,103.8900,1.088333,40,78,2024-01-19 04:22:54,662466
4,312564000,9218923,5,0,103.8913,1.087745,229,317,2024-01-21 04:06:11,662466


In [7]:
df.tail()

,MMSI,IMO,STATUS,SPEED,LON,LAT,COURSE,HEADING,TIMESTAMP,SHIP_ID
19069,533132848,9218923,5,24,101.2966,2.943483,34,31,2024-04-29 23:34:39,662466
19070,533132848,9218923,5,27,101.2974,2.944483,38,119,2024-04-29 23:36:49,662466
19071,533132848,9218923,5,9,101.2983,2.944467,123,133,2024-04-29 23:40:09,662466
19072,533132848,9218923,5,9,101.2988,2.944017,132,118,2024-04-29 23:43:08,662466
19073,533132848,9218923,5,1,101.2992,2.943683,149,122,2024-04-29 23:49:09,662466


In [8]:

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])


# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Calculate the distance
    distance = R * c
    return distance

# Calculate distance traveled per day
daily_distance = {}
prev_lat, prev_lon = np.nan, np.nan
prev_timestamp = None

for index, row in df.iterrows():
    curr_lat = row['LAT']
    curr_lon = row['LON']
    curr_timestamp = row['TIMESTAMP']
    
    if not pd.isnull(prev_lat) and not pd.isnull(prev_lon):
        # Calculate distance between consecutive points
        dist = haversine(prev_lat, prev_lon, curr_lat, curr_lon)
        
        # Check if it's a new day
        if prev_timestamp.date() != curr_timestamp.date():
            # Store daily distance for the previous day
            if prev_timestamp.date() not in daily_distance:
                daily_distance[prev_timestamp.date()] = 0.0
            daily_distance[prev_timestamp.date()] += dist
        else:
            # Add distance to the current day
            if curr_timestamp.date() not in daily_distance:
                daily_distance[curr_timestamp.date()] = 0.0
            daily_distance[curr_timestamp.date()] += dist
    
    # Update previous values
    prev_lat, prev_lon = curr_lat, curr_lon
    prev_timestamp = curr_timestamp

# Print daily distances
for date, distance in daily_distance.items():
    print(f"Date: {date}, Distance Traveled: {distance:.3f} km")


Date: 2024-01-19, Distance Traveled: 0.546 km
Date: 2024-01-21, Distance Traveled: 0.461 km
Date: 2024-01-22, Distance Traveled: 2.202 km
Date: 2024-01-23, Distance Traveled: 5.074 km
Date: 2024-01-24, Distance Traveled: 4.536 km
Date: 2024-01-25, Distance Traveled: 216.685 km
Date: 2024-01-26, Distance Traveled: 176.281 km
Date: 2024-01-27, Distance Traveled: 30.657 km
Date: 2024-01-28, Distance Traveled: 2.965 km
Date: 2024-01-29, Distance Traveled: 1.256 km
Date: 2024-01-30, Distance Traveled: 0.768 km
Date: 2024-01-31, Distance Traveled: 8.260 km
Date: 2024-02-01, Distance Traveled: 3.599 km
Date: 2024-02-02, Distance Traveled: 16.224 km
Date: 2024-02-03, Distance Traveled: 62.200 km
Date: 2024-02-04, Distance Traveled: 42.420 km
Date: 2024-02-05, Distance Traveled: 40.599 km
Date: 2024-02-06, Distance Traveled: 44.662 km
Date: 2024-02-07, Distance Traveled: 55.416 km
Date: 2024-02-08, Distance Traveled: 59.865 km
Date: 2024-02-09, Distance Traveled: 51.663 km
Date: 2024-02-10, Dis

In [43]:
groups = []  # List to store groups
current_group = []  # List to store positions of the current group
in_speed_zero_sequence = False  # Flag to track if in a sequence of SPEED=0

# Iterate through the DataFrame
for index, row in df.iterrows():
    if row['SPEED'] == 0:
        if not in_speed_zero_sequence:
            # Start a new group for SPEED=0 sequence
            if current_group:
                # Store the current group if it's not empty
                groups.append(current_group)
                current_group = []  # Reset current group
            # Add this position to the new group
            current_group.append(row)
            in_speed_zero_sequence = True
        else:
            # Continue adding to the current SPEED=0 sequence
            current_group.append(row)
    else:
        # Found SPEED > 0, end the current group
        if in_speed_zero_sequence:
            # Add this non-zero speed position to the group
            current_group.append(row)
            # Store the completed group
            groups.append(current_group)
            current_group = []  # Reset current group
            in_speed_zero_sequence = False

# Add the last group if it's not empty
if current_group:
    groups.append(current_group)



In [44]:
len(groups)

1643

In [71]:
groups[0]

[MMSI                   312564000
 IMO                      9218923
 STATUS                         5
 SPEED                          0
 LON                       103.89
 LAT                     1.088333
 COURSE                        40
 HEADING                      511
 TIMESTAMP    2024-01-19 03:16:53
 SHIP_ID                   662466
 Name: 0, dtype: object,
 MMSI                   312564000
 IMO                      9218923
 STATUS                         5
 SPEED                          0
 LON                       103.89
 LAT                     1.088333
 COURSE                        40
 HEADING                       77
 TIMESTAMP    2024-01-19 03:37:53
 SHIP_ID                   662466
 Name: 1, dtype: object,
 MMSI                   312564000
 IMO                      9218923
 STATUS                         5
 SPEED                          0
 LON                     103.8914
 LAT                     1.089367
 COURSE                        40
 HEADING                       7

In [72]:
groups[1]

[MMSI                   533132848
 IMO                      9218923
 STATUS                         5
 SPEED                          0
 LON                     103.8913
 LAT                     1.087833
 COURSE                       348
 HEADING                      319
 TIMESTAMP    2024-01-22 13:08:49
 SHIP_ID                   662466
 Name: 39, dtype: object,
 MMSI                   533132848
 IMO                      9218923
 STATUS                         5
 SPEED                          1
 LON                     103.8913
 LAT                      1.08785
 COURSE                       348
 HEADING                      319
 TIMESTAMP    2024-01-22 13:29:47
 SHIP_ID                   662466
 Name: 40, dtype: object]

In [61]:
data1=groups[10]

In [67]:
pd.to_datetime(data1[0]['TIMESTAMP']),pd.to_datetime(data1[-1]['TIMESTAMP'])

(Timestamp('2024-01-26 11:57:59'), Timestamp('2024-01-26 12:01:21'))

In [68]:
pd.to_datetime(data1[-1]['TIMESTAMP'])-pd.to_datetime(data1[0]['TIMESTAMP'])

Timedelta('0 days 00:03:22')

In [70]:
for group in groups:
    last_timestamp = pd.to_datetime(group[-1]['TIMESTAMP'])
    first_timestamp = pd.to_datetime(group[0]['TIMESTAMP'])
    print(last_timestamp-first_timestamp)


3 days 09:24:56
0 days 00:20:58
0 days 12:48:10
0 days 00:03:01
0 days 05:26:04
1 days 21:17:09
0 days 00:08:03
0 days 00:12:22
0 days 00:06:12
0 days 00:12:11
0 days 00:03:22
0 days 00:12:20
0 days 00:06:13
0 days 00:03:15
0 days 00:15:17
0 days 00:03:05
0 days 00:05:11
0 days 00:06:00
0 days 00:06:06
0 days 00:03:04
0 days 00:04:01
0 days 00:17:45
0 days 00:06:00
0 days 00:03:09
0 days 00:06:00
0 days 00:15:36
0 days 00:18:36
0 days 00:50:25
0 days 01:40:48
0 days 00:42:00
0 days 00:15:01
0 days 00:32:53
0 days 00:27:00
0 days 00:05:56
0 days 00:02:54
0 days 00:44:56
0 days 00:02:51
0 days 00:23:34
0 days 00:04:31
0 days 00:25:38
0 days 00:10:42
0 days 00:07:25
0 days 01:19:40
0 days 01:04:36
0 days 00:07:02
0 days 00:20:44
0 days 04:18:04
0 days 00:20:59
0 days 04:12:18
0 days 04:57:24
0 days 02:11:59
0 days 01:03:00
0 days 00:21:18
0 days 00:24:23
0 days 00:24:05
0 days 01:09:02
0 days 07:09:25
0 days 04:38:56
0 days 00:21:05
0 days 01:24:04
0 days 00:41:59
0 days 00:24:01
0 days 0